In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine


import talib as ta

In [5]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Functions

In [7]:
# 通过signal拿到了一个series position
def get_position(series,isShift=False):
    list_position = [0]
    # list_position.extend([0]*(len(roc_signal['signal'])))
    for i,signal in enumerate(series):   
        if signal == 1:
            list_position.append(1)
        elif signal == -1:
            list_position.append(0)
        else:
            list_position.append(list_position[-1])
            
    if isShift:
        return list_position
    else:
        return list_position[1:]

In [8]:
def day_to_week(df,isrolling=True,rollinig_period = 20,keywords='CLOSE'):
    """
    周度数据，高频变低频，均值-前值填充
    """
    if isrolling==True:
        df['rolling'] = df[keywords].rolling(rollinig_period).mean()
    else:
        df = df
    df.index = pd.to_datetime(df.index)
    df_ = df.resample('W-SUN').last().ffill().dropna()  # resample用前最邻近值
    return df_

In [9]:
# 位置原则，与给定值的距离
def positional_rule_value(df,indicator_name,isInverse=False,value1 = 0,value2 = 0):
    if isInverse:
        con_1 = df[indicator_name] < value1
        con_2 = df[indicator_name] > value2
    else:
        con_1 = df[indicator_name] > value1
        con_2 = df[indicator_name] < value2
    df.loc[:, 'signal'] = np.where(con_1, 1, np.where(con_2,-1,0))
    return df.loc[:, 'signal']

In [10]:
# 位置原则，参考线
def positional_rule_reference_line(df,indicator_name,reference_line):
    con_1 = df[reference_line] > df[indicator_name]
    con_2 = df[reference_line] < df[indicator_name]
    df.loc[:, 'signal'] = np.where(con_1, 1, np.where(con_2,-1,0))
    return df.loc[:, 'signal']

In [11]:
# 位置原则，与通道的距离
def positional_rule_bands(df,upperband,lowerband,reference_line,isMTM=True):
    con_1 = df[reference_line] > df[upperband]
    con_2 = df[reference_line] < df[lowerband]
    
    if isMTM:
        df.loc[:, 'signal'] = np.where(con_1, 1, np.where(con_2,-1,0))
    else:
        df.loc[:, 'signal'] = np.where(con_2, 1, np.where(con_1,-1,0))

    return df.loc[:, 'signal']

In [12]:
# dma_ama inclusive
def dma_ama_rule(df):
    # 根据sma判断买入卖出信号并写入（当天信号。当天卖出）
    con_1 = (df['dma'] > 0) & (df['ama'] > 0) & ((df['dma'] - df['dma'].shift(1)) > 0) & (
            (df['ama'] - df['ama'].shift(1)) > 0)
    con_2 = (df['dma'] < 0) & (df['ama'] < 0) & ((df['dma'] - df['dma'].shift(1)) < 0) & (
            (df['ama'] - df['ama'].shift(1)) < 0)
    df.loc[:, 'signal'] = np.where(con_1, 1, np.where(con_2, -1, 0))
    return df.loc[:,'signal']

In [13]:
# 交叉原则，与给定值交叉
def cross_rule(df,isValue=False,**kargs):
    def cross_rule_value(indicator_name,value1,value2):# 传入value1、value2
        con_1 = ((df[indicator_name].shift(1) < value1) & (df[indicator_name] > value1)) | ((df[indicator_name].shift(1) < value2) & (df[indicator_name] > value2))
        con_2 = ((df[indicator_name].shift(1) > value1) & (df[indicator_name] < value1)) | ((df[indicator_name].shift(1) > value2) & (df[indicator_name] < value2))
        df.loc[:, 'signal'] = np.where(con_1, 1, np.where(con_2, -1, 0))
    def cross_rule_reference_line(ReferenceLine1,ReferenceLine2):  # 传入ReferenceLine1，ReferenceLine2
        con_1 = (df[ReferenceLine1].shift(1) < df[ReferenceLine2].shift(1)) & (df[ReferenceLine1] > df[ReferenceLine2])  # 上穿买入，动量策略
        con_2 = (df[ReferenceLine1].shift(1) > df[ReferenceLine2].shift(1)) & (df[ReferenceLine1] < df[ReferenceLine2])
        df.loc[:, 'signal'] = np.where(con_1, 1, np.where(con_2, -1, 0))
    if not isValue:
        cross_rule_value(indicator_name=kargs['indicator_name'],value1=kargs['value1'],value2=kargs['value2'])
        return df.loc[:, 'signal']

    cross_rule_reference_line(ReferenceLine1=kargs['ReferenceLine1'],ReferenceLine2=kargs['ReferenceLine2'])
    return df.loc[:, 'signal']

In [14]:
def mom_rule(df,indicator_name='CLOSE',isMTM=True):
    if isMTM:
        con_1 = df[indicator_name] > df[indicator_name].shift(1)
        con_2 = df[indicator_name] < df[indicator_name].shift(1)
    else:
        con_1 = df[indicator_name] < df[indicator_name].shift(1)
        con_2 = df[indicator_name] > df[indicator_name].shift(1)
        
    df.loc[:, 'signal'] = np.where(con_1, 1, np.where(con_2, -1, 0))
    return df.loc[:,'signal']

In [15]:
def yoy_mtm_rule(df,indicator_name):
    df['yoy'] =df[indicator_name]/df[indicator_name].shift(12)
    con_1 = df['yoy'] > df['yoy'].shift(1)
    con_2 = df['yoy'] < df['yoy'].shift(1)
    df.loc[:,'signal'] = np.where(con_1, 1, np.where(con_2, -1, 0))
    return df.loc[:,'signal']

In [16]:
def consecutive_days_rule(df,indicator_name):
    df['return'] = df[indicator_name].pct_change(1)
    df['streak_up'] = df['return'].gt(0).astype(int).groupby(df['return'].le(0).astype(int).cumsum()).cumsum()  # gt(0)=1计数，gt(0)=0分组
    df['streak_down'] = df['return'].lt(0).astype(int).groupby(df['return'].ge(0).astype(int).cumsum()).cumsum()
    con_1 = df['streak_up'] >= 3
    con_2 = df['streak_down'] >= 3
    df.loc[:,'signal'] = np.where(con_1,1,np.where(con_2,-1,0))
    return df.loc[:,'signal']

In [17]:
from sqlalchemy import create_engine
engine = create_engine("mysql+pymysql://root:@localhost:3306/meta_data?charset=utf8mb4")

# 获取Wind全A周频率收盘价数据
df_881001 = pd.read_sql('SELECT Date,HIGH,LOW,CLOSE FROM 881001_wi', engine,index_col='Date')  # 从meta_data提取数据
df_881001_ = day_to_week(df_881001,isrolling=False)

In [18]:
def get_sma_idc(df, timeperiod=20):
    df['sma'] = ta.SMA(df['CLOSE'].values, timeperiod)
    return df.loc[:,'sma']

In [23]:
engine = create_engine("mysql+pymysql://root:@localhost:3306/factors_pool?charset=utf8mb4")

In [77]:
engine = create_engine("mysql+pymysql://root:@localhost:3306/factors_signal?charset=utf8mb4")

# 技术指标

In [20]:
print('***'*33)

***************************************************************************************************


## roc

In [20]:
roc_idc = pd.read_sql('SELECT * FROM roc_881001_w', engine,index_col='Date')  

In [21]:
roc_signal = positional_rule_value(roc_idc,'roc')

In [78]:
roc_signal.to_sql(name='roc',con=engine,if_exists='replace',index=True,index_label='Date')  # 因子值写入sql

1537

## sma

In [24]:
sma_idc = pd.read_sql('SELECT * FROM sma_881001_w', engine,index_col='Date')  

In [25]:
df_sma = pd.merge(left=sma_idc,right=df_881001_['CLOSE'],left_index=True,right_index=True,how='left')

In [26]:
sma_signal = positional_rule_reference_line(df_sma,'sma','CLOSE')

In [79]:
sma_signal.to_sql(name='sma',con=engine,if_exists='replace',index=True,index_label='Date')  # 因子值写入sql

1537

## dma

In [27]:
dma_idc = pd.read_sql('SELECT * FROM dma_881001_w', engine,index_col='Date')  

In [28]:
dma_signal = positional_rule_value(dma_idc,'dma')

In [81]:
dma_signal.to_sql(name='dma',con=engine,if_exists='replace',index=True,index_label='Date')  # 因子值写入sql

1537

## ama

In [29]:
dma_ama_idc = pd.read_sql('SELECT ama_881001_w.Date,ama,dma FROM ama_881001_w LEFT JOIN dma_881001_w ON ama_881001_w.Date=dma_881001_w.Date ', engine,index_col='Date')  

In [30]:
dma_ama_signal = dma_ama_rule(dma_ama_idc)

In [82]:
dma_signal.to_sql(name='dma_ama',con=engine,if_exists='replace',index=True,index_label='Date')  # 因子值写入sql

1537

## macd

In [31]:
macd_idc = pd.read_sql('SELECT * FROM macd_881001_w', engine,index_col='Date')  

In [32]:
macd_signal = positional_rule_value(macd_idc,'macd')

In [83]:
macd_signal.to_sql(name='macd',con=engine,if_exists='replace',index=True,index_label='Date')  # 因子值写入sql

1537

## trix

In [33]:
trix_idc = pd.read_sql('SELECT * FROM trix_881001_w', engine,index_col='Date')  

In [34]:
trix_signal = positional_rule_value(trix_idc,'trix_diff')

In [84]:
trix_signal.to_sql(name='trix',con=engine,if_exists='replace',index=True,index_label='Date')  # 因子值写入sql

1537

## bbi

In [35]:
bbi_idc = pd.read_sql('SELECT * FROM bbi_881001_w', engine,index_col='Date')  

In [36]:
df_bbi = pd.merge(left=bbi_idc,right=df_881001_['CLOSE'],left_index=True,right_index=True,how='left')

In [37]:
bbi_signal = positional_rule_reference_line(df_bbi,'bbi','CLOSE')

In [85]:
bbi_signal.to_sql(name='bbi',con=engine,if_exists='replace',index=True,index_label='Date')  # 因子值写入sql

1537

## bbands

In [38]:
bbands_idc = pd.read_sql('SELECT * FROM bbands_881001_w', engine,index_col='Date')  

In [39]:
bbands_signal = positional_rule_bands(bbands_idc,'upperband','lowerband','CLOSE')

In [86]:
bbands_signal.to_sql(name='bbands',con=engine,if_exists='replace',index=True,index_label='Date')  # 因子值写入sql

1537

## bias

In [40]:
bias_idc = pd.read_sql('SELECT * FROM bias_881001_w', engine,index_col='Date')  

In [41]:
bias_signal = positional_rule_value(bias_idc,'bias',value1=20,value2=-20)

In [87]:
bias_signal.to_sql(name='bias',con=engine,if_exists='replace',index=True,index_label='Date')  # 因子值写入sql

1537

## cci

In [42]:
cci_idc = pd.read_sql('SELECT * FROM cci_881001_w', engine,index_col='Date')  

In [43]:
cci_signal = cross_rule(cci_idc,indicator_name='cci',value1=100,value2=-100)

In [88]:
cci_signal.to_sql(name='cci',con=engine,if_exists='replace',index=True,index_label='Date')  # 因子值写入sql

1537

## KDJ

In [44]:
kdj_idc = pd.read_sql('SELECT * FROM kdj_881001_w', engine,index_col='Date')  

In [45]:
kdj_signal = cross_rule(kdj_idc,isValue=True,ReferenceLine1='slowk',ReferenceLine2='slowd')

In [89]:
kdj_signal.to_sql(name='kdj',con=engine,if_exists='replace',index=True,index_label='Date')  # 因子值写入sql

1537

## rsi

In [46]:
rsi_idc = pd.read_sql('SELECT * FROM rsi_881001_w', engine,index_col='Date') 

In [47]:
rsi_signal = positional_rule_value(rsi_idc,'rsi',isInverse=True,value1=20,value2=80)

In [90]:
rsi_signal.to_sql(name='rsi',con=engine,if_exists='replace',index=True,index_label='Date')  # 因子值写入sql

1537

## cmo

In [48]:
cmo_idc = pd.read_sql('SELECT * FROM cmo_881001_w', engine,index_col='Date') 

In [49]:
cmo_signal = positional_rule_value(cmo_idc,'cmo',isInverse=True,value1=0,value2=0)

In [91]:
cmo_signal.to_sql(name='cmo',con=engine,if_exists='replace',index=True,index_label='Date')  # 因子值写入sql

1537

# 宏观因子

## 银行间质押式回购加权利率:7天

In [50]:
zyhgitt_idc = pd.read_sql('SELECT * FROM zyhgitt_w', engine,index_col='Date') 

In [51]:
zyhgitt_signal = mom_rule(zyhgitt_idc,isMTM=False)

In [92]:
zyhgitt_signal.to_sql(name='zyhgitt',con=engine,if_exists='replace',index=True,index_label='Date')  # 因子值写入sql

1324

## 中长期贷款余额:同比

In [52]:
zcqlb_idc = pd.read_sql('SELECT * FROM zcqlb_m', engine,index_col='Date')

In [53]:
zcqlb_signal = mom_rule(zcqlb_idc,isMTM=True)

In [93]:
zcqlb_signal.to_sql(name='zcqlb',con=engine,if_exists='replace',index=True,index_label='Date')  # 因子值写入sql

365

## 制造业PMI:新出口订单

In [54]:
xckpmiidy_idc = pd.read_sql('SELECT * FROM xckpmiidy_m', engine,index_col='Date')

In [55]:
xckpmiidy_signal = mom_rule(xckpmiidy_idc,isMTM=True)

In [94]:
xckpmiidy_signal.to_sql(name='xckpmiidy',con=engine,if_exists='replace',index=True,index_label='Date')  # 因子值写入sql

233

## 社会融资规模存量:同比

In [56]:
srclyoy_idc = pd.read_sql('SELECT * FROM srclyoy_m', engine,index_col='Date')

In [57]:
srclyoy_signal = mom_rule(srclyoy_idc,isMTM=True)

In [95]:
srclyoy_signal.to_sql(name='srclyoy',con=engine,if_exists='replace',index=True,index_label='Date')  # 因子值写入sql

233

## 中美十年国债收益率_差值

In [58]:
tb10y_diff_idc = pd.read_sql('SELECT * FROM tb10y_diff_w', engine,index_col='Date')

In [59]:
sma_tb10y_diff = get_sma_idc(tb10y_diff_idc,timeperiod=20)
df_tb10y_diff = tb10y_diff_idc.copy()

In [60]:
tb10y_diff_signal = positional_rule_reference_line(df_tb10y_diff,indicator_name='sma',reference_line='CLOSE')

In [96]:
tb10y_diff_signal.to_sql(name='tb10y_diff',con=engine,if_exists='replace',index=True,index_label='Date')  # 因子值写入sql

1168

## 新成立基金份额

In [61]:
nrfs_idc = pd.read_sql('SELECT * FROM nrfs_w', engine,index_col='Date')

In [62]:
sma_nrfs = get_sma_idc(nrfs_idc,timeperiod=20)
df_nrfs = nrfs_idc.copy()

In [63]:
nrfs_signal = positional_rule_reference_line(df_nrfs,indicator_name='sma',reference_line='CLOSE')

In [97]:
nrfs_signal.to_sql(name='nrfs',con=engine,if_exists='replace',index=True,index_label='Date')  # 因子值写入sql

1205

## 股债性价比

In [64]:
tb10y_ttm_idc = pd.read_sql('SELECT * FROM tb10ymttm_w', engine,index_col='Date')

In [65]:
tb10y_ttm_idc['lowerband'] = tb10y_ttm_idc['CLOSE'].rolling(52).mean() - 2 * (tb10y_ttm_idc['CLOSE'].rolling(52).std())
tb10y_ttm_idc['upperband'] = tb10y_ttm_idc['CLOSE']*2
df_tb10y_ttm = tb10y_ttm_idc.copy()

In [66]:
tb10y_ttm_signal = positional_rule_bands(df_tb10y_ttm,upperband='lowerband',lowerband='lowerband',reference_line='CLOSE',isMTM=False)

In [98]:
tb10y_ttm_signal.to_sql(name='tb10y_ttm',con=engine,if_exists='replace',index=True,index_label='Date')  # 因子值写入sql

1169

# 情绪指标

## rzmr

In [67]:
bbands_rzmr_idc = pd.read_sql('SELECT * FROM bbands_rzmr_w', engine,index_col='Date')

In [68]:
bbands_rzmr_signal = positional_rule_bands(bbands_rzmr_idc,'upperband','lowerband','CLOSE')

In [99]:
bbands_rzmr_signal.to_sql(name='bbands_rzmr',con=engine,if_exists='replace',index=True,index_label='Date')  # 因子值写入sql

739

## ic_cfe

In [69]:
bbands_iccfe_idc = pd.read_sql('SELECT * FROM bbands_iccfe_w', engine,index_col='Date')

In [70]:
bbands_iccfe_signal = positional_rule_bands(bbands_iccfe_idc,'upperband','lowerband','CLOSE')

In [100]:
bbands_iccfe_signal.to_sql(name='bbands_iccfe',con=engine,if_exists='replace',index=True,index_label='Date')  # 因子值写入sql

476

## if_cfe

In [71]:
bbands_ifcfe_idc = pd.read_sql('SELECT * FROM bbands_ifcfe_w', engine,index_col='Date')

In [72]:
bbands_ifcfe_signal = positional_rule_bands(bbands_ifcfe_idc,'upperband','lowerband','CLOSE')

In [101]:
bbands_ifcfe_signal.to_sql(name='bbands_ifcfe',con=engine,if_exists='replace',index=True,index_label='Date')  # 因子值写入sql

737

## vol

In [73]:
volumeratio_idc = pd.read_sql('SELECT * FROM volumeratio_w', engine,index_col='Date')

In [74]:
volumeratio_signal = consecutive_days_rule(volumeratio_idc,'volumeratio_')

In [102]:
volumeratio_signal.to_sql(name='volumeratio',con=engine,if_exists='replace',index=True,index_label='Date')  # 因子值写入sql

486

## oi

In [75]:
oiratio_idc = pd.read_sql('SELECT * FROM oiratio_m', engine,index_col='Date')

In [76]:
oiratio_signal = yoy_mtm_rule(oiratio_idc,'oiratio')

In [103]:
oiratio_signal.to_sql(name='oiratio',con=engine,if_exists='replace',index=True,index_label='Date')  # 因子值写入sql

112